<b>Notebook for exploring tourism intensity and other spatial attributes based on Inside Airbnb listing data </b>

In [ ]:
import pandas as pd
import datetime
import geopandas as gpd
import seaborn as sns
import glob as glob
import matplotlib.pyplot as plt
from pylab import savefig
import matplotlib.ticker as mtick
from scripts.data_wrangling import listing2gdf, census2gdf,CalculateTouristIntensity,aggregate
from mpl_toolkits.axes_grid1 import make_axes_locatable

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [ ]:
#listing data urls
apr2019 = "http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/2019-04-08/data/listings.csv.gz"
apr2018 = "http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/2018-04-07/data/listings.csv.gz"
apr2017 = "http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/2017-04-02/data/listings.csv.gz"
apr2016 = "http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/2016-04-04/data/listings.csv.gz"
apr2015 = "http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/2015-04-05/data/listings.csv.gz"

listings_2015_2019 = [apr2015,apr2019,apr2018,apr2017,apr2016]

In [ ]:
#retrieve and process that listing data
for url in listings_2015_2019:
    
    date = url.split('/')[-3]
    year = date[0:4]
    
    print("Processing: \n" + url)
    
    #load airbnb data and seperate features
    airbnb_gdf = listing2gdf(url)
    room = airbnb_gdf.loc[airbnb_gdf['room_type'].isin(['Private room','Shared room'])]
    entire_home = airbnb_gdf.loc[airbnb_gdf['room_type']=='Entire home/apt']
    superhost = airbnb_gdf.loc[airbnb_gdf['host_is_superhost']=='t']
    illegal = airbnb_gdf.loc[airbnb_gdf['availability_365']>30]
    
    #loading Amsterdam census data
    nbh_gdf = census2gdf("./data/amsterdam_neighbourhoods.geojson")
    
    #aggregate airbnb features and census data
    nbh_gdf = aggregate(airbnb_gdf,nbh_gdf,room,entire_home,superhost,illegal)
    
    #calculate tourist intensity
    nbh_gdf = CalculateTouristIntensity(nbh_gdf,year)
    nbh_gdf['date'] = date
    
    #export listing and airbnb data as geojson files
    nbh_gdf.to_file("./output/AirbnbPoly_{}.geojson".format(date.replace('-','')), driver="GeoJSON",encoding='utf-8')
    airbnb_gdf.to_file("./output/AirbnbPoints_{}.geojson".format(date.replace('-','')),driver="GeoJSON",encoding='utf-8')
    
    print("Done!")

In [ ]:
#concatenate geojson from different years
data_poly = []
for poly in glob.glob("./output/AirbnbPoly_*"):
    gdf = gpd.read_file(poly, driver='GeoJSON')
    pdf = pd.DataFrame(gdf)
    data_poly.append(pdf)
    concat = pd.concat(data_poly, axis=0)
    combined_gdf = gpd.GeoDataFrame(concat)
combined_gdf.to_file("./output/AirbnbPoly_Concat_2015to2019.geojson", driver="GeoJSON",encoding='utf-8')

# data_point = []
# for poly in glob.glob("./output/AirbnbPoints_*"):
#     gdf = gpd.read_file(poly, driver='GeoJSON')
#     pdf = pd.DataFrame(gdf)
#     data_point.append(pdf)
#     concat = pd.concat(data_point, axis=0)
#     combined_gdf = gpd.GeoDataFrame(concat)
# combined_gdf.to_file("./output/AirbnbPoints_Concat_2015to2019.geojson", driver="GeoJSON",encoding='utf-8')

<b>Plots </b>

In [ ]:
poly = gpd.read_file('./output/AirbnbPoly_Concat_2015to2019.geojson', driver='GeoJSON')

In [ ]:
poly.head(1)

In [ ]:
data2018 = listing = poly[poly['date']=='2018-04-07']
    
fig, ax = plt.subplots(figsize=(10,10))
rdnew_crs = {'init' :'epsg:28992'}

ax.set_title("Number of illegal listing (30+ days) Amsterdam 2018-04-07", fontsize=12,fontweight='bold')
data2018.to_crs(rdnew_crs).dropna(subset = ['Airbnb_Over30Days']).plot(column='Airbnb_Over30Days', ax=ax,cmap='Greens')
data2018.to_crs(rdnew_crs).dropna(subset = ['Airbnb_Over30Days']).geometry.boundary.plot(color=None,edgecolor='black',linewidth=0.1,ax=ax)

#ax.axis('off')
vmin = data2018['Airbnb_Over30Days'].min()
vmax = data2018['Airbnb_Over30Days'].max()

# Create colorbar
sm = plt.cm.ScalarMappable(cmap='Greens', norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm._A = []

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
cbar = fig.colorbar(sm,cax=cax)
ax.set(xlabel="Easting", ylabel="Northing")
ax.annotate("Source: Municipality Amsterdam;Inside Airbnb, 2018",xy=(0.14, .16), xycoords='figure fraction', horizontalalignment='left', verticalalignment='bottom', fontsize=10, color='#555555')
plt.savefig("./images/IllegalApril2018.png",bbox_inches = "tight",dpi=300)

In [ ]:
#Top 10 Airbnb Neighbourhoods in Amsterdam April 2015-2019
for date in poly['date'].unique():
    listing = poly[poly['date'].str.contains(date)]
    listing = listing.sort_values(by='Airbnb_ListingCount',ascending=False)[:10]
    sns.set(style="darkgrid")
    
    fig, ax = plt.subplots(ncols=2)
    fig.suptitle("Top 10 Airbnb Neighourhoods in \nAmsterdam {}".format(date),size=12,weight='bold',y=1.02,x=0.03,ha='left')
    
    sns.barplot(x='Airbnb_RoomRentalCount', y='Buurt', data=listing, color="#F7F48B",ax=ax[0])
    ax[0].set_title("Room")
    ax[0].set(xlabel="                                       Airbnb Listing", 
              ylabel="Neighbourhood",xlim=(0, 300)) 
    
    ax[0].invert_xaxis()
    
    sns.barplot(x='Airbnb_EntireLodgeCount', y='Buurt',data= listing, color="#f49191",ax=ax[1])
    ax[1].set_title("Entire Lodge")
    ax[1].set(xlabel="", ylabel="",xlim=(0, 300))
    ax[1].set_yticks([])
    #plt.tight_layout(w_pad=-2.5)
    sns.despine(left=True, bottom=False)
    plt.subplots_adjust(wspace=0)
    plt.savefig("./images/top10Pricey_april{}.png".format(date[0:4]),bbox_inches = "tight",dpi=300)

In [ ]:
#Top 10 Expensive Airbnb Neighbourhoods in Amsterdam April 2015-2019
for date in poly['date'].unique():
    listing = poly[poly['date'].str.contains(date)]
    listing = listing.sort_values(by='Airbnb_AvgPriceRoom',ascending=False)[:10]
    sns.set(style="darkgrid")
    
    fig, ax = plt.subplots(ncols=2)
    fig.suptitle("Top 10 Expensive Airbnb Neighbourhoods in Amsterdam {}".format(date),size=12,weight='bold',y=1.02,x=0.03,ha='left')
    
    sns.barplot(x='Airbnb_AvgPriceRoom', y='Buurt', data=listing, color="#F7F48B",ax=ax[0])
    ax[0].set_title("Room")
    ax[0].set(xlabel="                                       Average Price", 
              ylabel="Neighbourhood",xlim=(0, 500)) 
    
    ax[0].invert_xaxis()
    
    sns.barplot(x='Airbnb_AvgPriceEntireLodge', y='Buurt',data= listing, color="#f49191",ax=ax[1])
    ax[1].set_title("Entire Lodge")
    ax[1].set(xlabel="", ylabel="",xlim=(0, 500))
    ax[1].set_yticks([])
    sns.despine(left=True, bottom=False)
    fmt = '$%.0f'
    tick = mtick.FormatStrFormatter(fmt)
    ax[1].xaxis.set_major_formatter(tick)
    ax[0].xaxis.set_major_formatter(tick)
    sns.despine(left=True, bottom=False)
    plt.subplots_adjust(wspace=0)
    plt.savefig("./images/top10Pricey_april{}.png".format(date[0:4]),bbox_inches = "tight",dpi=300)

In [ ]:
point = gpd.read_file("./output/AirbnbPoints_Concat_2015to2019.geojson",driver='GeoJSON')

In [ ]:
data = []
over30days = []
for date in point['date'].unique():
    listing = point[point['date'].str.contains(date)]
    days30 = len(listing[listing['availability_365']>30])/len(listing) * 100
    data.append(date)
    over30days.append(days30)
df = pd.DataFrame(list(zip(data, over30days)), columns =['date', 'over30days_rate'])
df['over30days_rate']= df['over30days_rate'].round(2)